# Converting Our MP_DATA numpy dataset

### NOTE: This should only be run by Shelby or people who have the original dataset of jpg files

In [2]:
import cv2
import keyboard
import os
import numpy as np
import time
from PredictLetter import mp_detection, formatPoints, draw_styled_landmarks, mp_hands

#### create array to signify the actions for iterating

# actions to be detected by model
actions = np.array(['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])



#### create all the path variables

# path for exported data (numpy arrays)
DATA_PATH = os.path.join('MP_DATA')

# the path for the data used to train our model
TRAIN_PATH = os.path.join(DATA_PATH, 'asl_alphabet_train')

# the path for the data used to test our model
TEST_PATH = os.path.join(DATA_PATH, 'asl_alphabet_test')

# the path of the downloaded dataset folder holding all our images to extract the points from
DATASET_PATH = 'D:\\archive\\ASL_Alphabet_Dataset'

# path to T folder in jpg dataset
ASL_T_PATH = os.path.join(DATASET_PATH, 'asl_alphabet_train', 'T')

# path to P folder in jpg dataset
ASL_P_PATH = os.path.join(DATASET_PATH, 'asl_alphabet_train', 'P')

# path to N folder in jpg dataset
ASL_N_PATH = os.path.join(DATASET_PATH, 'asl_alphabet_train', 'N')

### 1. Some of the images in the dataset we were working with are incorrect meaning we will have to fill the image dataset with proper images

##### Letter: T, N, P

In [ ]:
cap = cv2.VideoCapture(0)

numImagesNeeded = 300
while cap.isOpened():
    ret, image = cap.read()
    if keyboard.is_pressed("c"):
        cv2.imwrite(os.path.join(ASL_T_PATH, f"T_T_{numImagesNeeded}.jpg"), image)
        numImagesNeeded = numImagesNeeded - 1
        time.sleep(0.1)

    if numImagesNeeded == 0:
        break
    
    cv2.imshow('OpenCV Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()   

## Setup folders for converted dataset

In [3]:
#### create the folders for testing and training in user directory

# create the test folder
try:
    os.makedirs(TEST_PATH)
except:
    pass

# create the train folder with all folders for alphabet letters
for action in actions:
    try: 
        os.makedirs(os.path.join(TRAIN_PATH, action))
    except:
        pass

# Extracting points from our image dataset into our converted dataset

In [4]:
with mp_hands.Hands(max_num_hands=2, model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    for action in actions:
        CURR_PATH = os.path.join(DATASET_PATH, 'asl_alphabet_train', action)
        print(f"Extraction starting {action}")

        # temporary count variable
        # count = 0
        # the number of files we want to extract and store
        # maxFilesProcessed = 4000
        for filename in os.listdir(CURR_PATH):
            # if count >= maxFilesProcessed:
            #     break
            # grab the path of the current image in the dataset to extract points from
            IMG_PATH = os.path.join(CURR_PATH, filename)
            
            # read the image into a variable
            img = cv2.imread(IMG_PATH)
    
            # extract the keypoints from the image
            # original image
            image, results1 = mp_detection(img, hands)

            # flipped image for opposite hand
            image2, results2 = mp_detection(cv2.flip(img,1), hands)

            # store in array to iterate through
            res = [results1, results2]

            uniqueEnd = "orig"
            for results in res:
                if results.multi_hand_landmarks and len(results.multi_handedness) == 1:
                    # format the points 
                    points = formatPoints(results)
            
                    # path to the correct folder to save the points
                    SAVE_PATH = os.path.join(TRAIN_PATH, action, filename + uniqueEnd)
            
                    # save the points to the correct folder
                    np.save(SAVE_PATH, points)
                    # count += 1
                uniqueEnd = "flip"
            
        print(f"Extraction complete {action}")

Extraction starting A
Extraction complete A
Extraction starting B
Extraction complete B
Extraction starting C
Extraction complete C
Extraction starting D
Extraction complete D
Extraction starting del
Extraction complete del
Extraction starting E
Extraction complete E
Extraction starting F
Extraction complete F
Extraction starting G
Extraction complete G
Extraction starting H
Extraction complete H
Extraction starting I
Extraction complete I
Extraction starting J
Extraction complete J
Extraction starting K
Extraction complete K
Extraction starting L
Extraction complete L
Extraction starting M
Extraction complete M
Extraction starting N
Extraction complete N
Extraction starting nothing
Extraction complete nothing
Extraction starting O
Extraction complete O
Extraction starting P
Extraction complete P
Extraction starting Q
Extraction complete Q
Extraction starting R
Extraction complete R
Extraction starting S
Extraction complete S
Extraction starting space
Extraction complete space
Extracti

In [3]:
# our "nothing" data folder was messed up due to improper conversion so we are going to fill the nothing folder with empty numpy arrays

for num in range(10000):
    SAVE_PATH = os.path.join(TRAIN_PATH, "nothing", str(num))
    np.save(SAVE_PATH, np.zeros(21*3*2))